# SIR

In [277]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go 
import plotly.express as px 

## Deterministic Model

The dynamic of the SIR model:
\\[
\begin{aligned}
\frac{dS}{dt} &= -\frac{\beta SI}{N} \\
\frac{dI}{dt} &= \frac{\beta SI}{N} - \gamma I\\
\frac{dR}{dt} &= \gamma I 
\end{aligned}
\\]

In [148]:
T = 1000
β = 0.02
γ = 0.01
N = 1000

S = np.zeros(T+1)
I = np.zeros(T+1)
R = np.zeros(T+1)

S[0], I[0] = N-10, 10

In [149]:
for t in range(T-1):
    a, b = β*S[t]*I[t]/N, γ*I[t]
    S[t+1] = S[t] - a
    I[t+1] = I[t] + a - b
    R[t+1] = R[t] + b

In [150]:
fig = go.Figure()
fig.add_scatter(x=np.arange(T), y=S, name="Susceptible", hovertemplate="%{y:.0f}")
fig.add_scatter(x=np.arange(T), y=I, name="Infected", hovertemplate="%{y:.0f}")
fig.add_scatter(x=np.arange(T), y=R, name="Removed", hovertemplate="%{y:.0f}")
fig.show()

## Gillespie Algorithm

In [302]:
T = 1000
β = 0.02
γ = 0.01
N = 1000

t = 0.
S = N - 10
I = 10
R = 0
data = []
np.random.seed(0)

In [303]:
while t < T:
    w1, w2 = β*S*I/N, γ*I
    w = w1 + w2
    t += -np.log(np.random.rand()) / w
    if np.random.rand() < w1/w:
        S -= 1
        I += 1
    else:
        I -= 1
        R += 1
    data.append((t, S, I, R))

In [304]:
t, S, I, R = zip(*data)

In [305]:
fig = go.Figure()
fig.add_scatter(x=t, y=S, name="Susceptible", hovertemplate="%{y:.0f}")
fig.add_scatter(x=t, y=I, name="Infected", hovertemplate="%{y:.0f}")
fig.add_scatter(x=t, y=R, name="Removed", hovertemplate="%{y:.0f}")
fig.show()

## Stochastic Model

The probability of each susceptible getting infected follows the exponential distribution:
\\[
\Pr(S \rightarrow I \text{ before Time } t|I)
= 1 - \exp \left(-\lambda t\right),
\\]
where $\lambda = \frac{\beta I}{N}$.

The probability of each infected getting removed follows the exponential distribution:
\\[
\Pr(I \rightarrow R \text{ before Time } t) = 1 - \exp(-\gamma t).
\\]

In [216]:
class Citizen:
    def __init__(self, ID, status=0, t_infected=None, t_removed=None):
        self.ID = ID
        self.status = status
        self.t_infected = t_infected
        self.t_removed = t_removed
        
    def __str__(self):
        s = str(self.ID) + ","
        if self.issusceptible():
            s += "Susceptible"
        elif self.isinfected():
            s += "Infected"
        elif self.isremoved():
            s += "Removed"
        s += "," + str(self.t_infected) + "," + str(self.t_removed)
        return s
        
    def issusceptible(self):
        return self.status == 0
    
    def isinfected(self):
        return self.status == 1
    
    def isremoved(self):
        return self.status == 2

In [217]:
class City:
    def __init__(self, citizens):
        self.citizens = citizens
        
    def __str__(self):
        s = ""
        for x in self.citizens:
            s += str(x) + "\n"
        return s
    
    def num_susceptible(self):
        return sum(x.issusceptible() for x in self.citizens)
        
    def num_infected(self):
        return sum(x.isinfected() for x in self.citizens)
        
    def num_removed(self):
        return sum(x.isremoved() for x in self.citizens) 

In [218]:
T = 1000
β = 0.02
γ = 0.01
N = 1000

S = np.zeros(T+1)
I = np.zeros(T+1)
R = np.zeros(T+1)

In [219]:
citizens = [Citizen(i) for i in range(N)]
for i in range(10):
    citizens[i].status = 1
    citizens[i].t_infected = 0
c = City(citizens)

In [220]:
np.random.seed(0)
for t in range(T):
    S[t] = c.num_susceptible()
    I[t] = c.num_infected()
    R[t] = c.num_removed()
    λ = β*I[t]/N
    for x in c.citizens:
        rand = np.random.rand()
        if x.issusceptible() and rand < 1 - np.exp(-λ):
            x.status = 1
            x.t_infected = t
        elif x.isinfected() and rand < 1 - np.exp(-γ):
            x.status = 2
            x.t_removed = t   
S[T] = c.num_susceptible()
I[T] = c.num_infected()
R[T] = c.num_removed()

In [221]:
fig = go.Figure()
fig.add_scatter(x=np.arange(T), y=S, name="Susceptible", hovertemplate="%{y:.0f}")
fig.add_scatter(x=np.arange(T), y=I, name="Infected", hovertemplate="%{y:.0f}")
fig.add_scatter(x=np.arange(T), y=R, name="Removed", hovertemplate="%{y:.0f}")
fig.show()

## Animation

In [399]:
T = 2000
β = 0.02
γ = 0.01
N = 10000

S = np.zeros(T+1)
I = np.zeros(T+1)
R = np.zeros(T+1)

np.random.seed(0)
citizens = [Citizen(i) for i in range(N)]
for _ in range(10):
    i = np.random.randint(N)
    citizens[i].status = 1
    citizens[i].t_infected = 0
citizens[np.random.randint(N)].status = 2
c = [City(citizens)]

In [400]:
np.random.seed(0)
for t in range(T):
    S[t] = c[t].num_susceptible()
    I[t] = c[t].num_infected()
    R[t] = c[t].num_removed()
    λ = β*I[t]/N
    citizens = []
    for i, x in enumerate(c[t].citizens):
        rand = np.random.rand()
        if x.issusceptible() and rand < 1 - np.exp(-λ):
            citizens.append(Citizen(i, 1, t))
        elif x.isinfected() and rand < 1 - np.exp(-γ):
            citizens.append(Citizen(i, 2, c[t].citizens[i].t_infected, t))
        else:
            citizens.append(c[t].citizens[i])
    c.append(City(citizens))
S[T] = c[T].num_susceptible()
I[T] = c[T].num_infected()
R[T] = c[T].num_removed()
cities = c

In [ ]:
fig = go.Figure(
        data=[go.Heatmap(
            x=np.arange(100), 
            y=np.arange(100),
            z=np.array([x.status for x in c[T].citizens]).reshape((100, 100)),
            colorscale=["lightyellow", "magenta", "lightgreen"])]
)
fig.show()

In [ ]:
fig = go.Figure(
    data=[go.Heatmap(
        x=np.arange(100), 
        y=np.arange(100),
        z=np.array([x.status for x in cities[0].citizens]).reshape((100, 100)),
        colorscale=["lightyellow", "magenta", "lightgreen"])],
    layout=go.Layout(
        updatemenus=[dict(type="buttons",
                          buttons=[dict(label="Play",
                                        method="animate",
                                        args=[None])])]),
    frames=[go.Frame(
        data=[go.Heatmap(
            x=np.arange(100), 
            y=np.arange(100),
            z=np.array([x.status for x in c.citizens]).reshape((100, 100)),
            colorscale=["lightyellow", "magenta", "lightgreen"])])
        for i, c in enumerate(cities) if i%10==0]
)        
fig.show()